In [34]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [35]:
df = pd.read_csv("../dataset/2012_2019_PlatteRiverWeir_features_merged_all.csv")
df.head()

,Unnamed: 0,SensorTime,CaptureTime,Filename,Agency,SiteNumber,TimeZone,Stage,Discharge,CalcTimestamp,...,WeirPt2X,WeirPt2Y,WwRawLineMin,WwRawLineMax,WwRawLineMean,WwRawLineSigma,WwCurveLineMin,WwCurveLineMax,WwCurveLineMean,WwCurveLineSigma
0,0,2012-06-09 13:15:00,2012-06-09T13:09:07,StateLineWeir_20120609_Farrell_001.jpg,USGS,6674500,MDT,2.99,916.0,2020-03-11T16:58:28,...,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2012-06-09 13:15:00,2012-06-09T13:10:29,StateLineWeir_20120609_Farrell_002.jpg,USGS,6674500,MDT,2.99,916.0,2020-03-11T16:58:33,...,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2012-06-09 13:45:00,2012-06-09T13:44:01,StateLineWeir_20120609_Farrell_003.jpg,USGS,6674500,MDT,2.96,873.0,2020-03-11T16:58:40,...,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2012-06-09 14:45:00,2012-06-09T14:44:30,StateLineWeir_20120609_Farrell_004.jpg,USGS,6674500,MDT,2.94,846.0,2020-03-11T16:58:47,...,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2012-06-09 15:45:00,2012-06-09T15:44:59,StateLineWeir_20120609_Farrell_005.jpg,USGS,6674500,MDT,2.94,846.0,2020-03-11T16:58:55,...,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
image_ds_tmp = tf.data.Dataset.list_files(
    '../dataset/images/*', shuffle='True')


In [37]:
for i in image_ds_tmp.take(5):
    print(i.numpy())

b'../dataset/images/StateLineWeir_20150411_Farrell_831.JPG'
b'../dataset/images/StateLineWeir_20170625_Farrell_180.jpg'
b'../dataset/images/StateLineWeir_20190223_Farrell_945.jpg'
b'../dataset/images/StateLineWeir_20170614_Farrell_867.jpg'
b'../dataset/images/StateLineWeir_20161117_Farrell_069.jpg'


In [38]:
IMG_SIZE = 512

In [110]:
from glob import glob
import random

def make_dataset(path, batch_size, df):

  def parse_image(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    #image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image

  def configure_for_performance(ds):
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.repeat()
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

  filenames = glob(path + '/*')
  random.shuffle(filenames)

  #labels = [classes.index(name.split('/')[-2]) for name in filenames]
  stage_values = [df[df.Filename == file.split('/')[-1]].Stage.values for file in filenames]
  discharge_values = [df[df.Filename == file.split('/')[-1]].Discharge.values for file in filenames]

  stage_discharge_values = [[np.squeeze(s), np.squeeze(d)] for s, d in zip(stage_values, discharge_values)]

  filenames_ds = tf.data.Dataset.from_tensor_slices(filenames)
  images_ds = filenames_ds.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  stage_discharge_ds = tf.data.Dataset.from_tensor_slices(stage_discharge_values)
  ds = tf.data.Dataset.zip((images_ds, stage_discharge_ds))
  ds = configure_for_performance(ds)

  return ds

In [111]:
path = "../dataset/images_tmp"

ds = make_dataset(path, 32, df)

In [112]:
for image, stage_discharge in ds.take(1):
    print(image.numpy().shape)
    print(stage_discharge.numpy().shape)

(32, 512, 512, 3)
(32, 2)
